In [10]:
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

import strategies.strategy_a as strategy_a
import strategies.strategy_b as strategy_b

# List of strategy modules
strategies = [strategy_a, strategy_b]

# Download real historical data using yfinance
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2023-01-01"
print(f"Downloading data for {ticker} from {start_date} to {end_date}...")
data = yf.download(ticker, start=start_date, end=end_date)
# returns = data['Close'].pct_change().dropna()
returns = pd.Series(data['Close'].dropna().values.squeeze(-1))
print("RETURNS: ",returns.pct_change().std(), '\nVol: ',returns.std())
# Use closing prices as a list for backtesting
# prices = data['Close'].values.tolist()
prices = list(data['Close'].to_numpy().squeeze(-1))

pcts = data


/var/folders/_v/y6tzlp694c59fn1z7bdq1x240000gn/T/ipykernel_17221/2906721908.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

RETURNS:  0.023265822826194358 
Vol:  30.421461997802503


In [15]:

def backtest_strategy(strategy_module, prices):
    # Get strategy name or fallback
    strategy_name = getattr(strategy_module, "NAME", "Unnamed Strategy")

    print(f"Running strategy: {strategy_name}")

    signals = strategy_module.run(prices)

    balance = 0
    position = 0
    pnl = []

    for i in range(1, len(prices)):
        if signals[i - 1] == "buy":
            position += 1
            balance -= prices[i]
        elif signals[i - 1] == "sell" and position > 0:
            position -= 1
            balance += prices[i]

        current_pnl = balance + position * prices[i]
        pnl.append(current_pnl)

    # Save results folder
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_path = os.path.join(results_dir, f"{strategy_name}_{timestamp}.png")

    plt.plot(pnl)
    plt.title(f"P&L Over Time - {strategy_name}")
    plt.xlabel("Time Step")
    plt.ylabel("P&L")
    plt.savefig(plot_path)
    plt.close()

    final_pnl = pnl[-1] if pnl else 0
    total_trades = sum(1 for s in signals if s in ["buy", "sell"])

    return final_pnl, total_trades

# Run backtests and collect summary
summary = []
for strat in strategies:
    final_pnl, total_trades = backtest_strategy(strat, prices)
    strategy_name = getattr(strat, "NAME", "Unnamed Strategy")
    summary.append({
        "Strategy": strategy_name,
        "Final P&L": final_pnl,
        "Total Trades": total_trades
    })

# Print summary table
df = pd.DataFrame(summary)
print(df)


Running strategy: buy increase, sell decrease
Running strategy: buy even days, sell odd days
                       Strategy   Final P&L  Total Trades
0   buy increase, sell decrease  586.851990           755
1  buy even days, sell odd days  -11.598824           756
